First I import data

In [3]:
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 423B-1A53

 Directory of c:\Users\77479\OneDrive\Documents\Python Scripts\Trivago

16/03/2024  12:36    <DIR>          .
15/03/2024  00:00    <DIR>          ..
16/03/2024  01:03        79,215,624 202303_Task1_Sessions.csv
14/03/2024  23:58        35,992,927 202303_Task1_Sessions.xlsx
16/03/2024  13:59            10,822 Case.ipynb
15/03/2024  00:00            91,291 Data structure.pdf
16/03/2024  12:34            57,411 Data_Analyst_-_Data_Intelligence_11.2023.pdf
               5 File(s)    115,368,075 bytes
               2 Dir(s)  276,513,955,840 bytes free


In [4]:
data = pd.read_csv('202303_Task1_Sessions.csv', delimiter=',')

In [6]:
data.head()

,"ymd,session_id,tracking_id,platform,is_app,is_repeater,traffic_type,country_name,agent_id,clickouts,bookings,session_duration,entry_page,total_ctp,arrival_day,departure_day"
0,"20220626,2022062620046057322,FA6JXA8TAJ,UK,0,1..."
1,"20220518,2022051821943006017,0X7RLU6KF7,BR,0,0..."
2,"20220508,2022050821020053928,0I59VWLQW0,UK,0,0..."
3,"20220507,2022050706015039122,JXNHOBQL50,CH,0,0..."
4,"20220523,2022052320052048087,W24I0V5Z2L,IT,0,0..."


Data didn't import correctly, so I'll adjust it

In [58]:
df = data[data.columns[0]].str.split(',', expand=True)
df = df.loc[:, :15]
df.columns = data.columns.str.split(',')[0]

# Descriptive analysis

In [54]:
df.head()

,ymd,session_id,tracking_id,platform,is_app,is_repeater,traffic_type,country_name,agent_id,clickouts,bookings,session_duration,entry_page,total_ctp,arrival_day,departure_day
0,20220626,2022062620046057322,FA6JXA8TAJ,UK,0,1,2,United Kingdom,16,0,0,29,2111,0,\N,\N
1,20220518,2022051821943006017,0X7RLU6KF7,BR,0,0,2,Brazil,2,3,0,1485,2100,27,20220530,20220531
2,20220508,2022050821020053928,0I59VWLQW0,UK,0,0,2,United Kingdom,20,0,0,143,2100,0,\N,\N
3,20220507,2022050706015039122,JXNHOBQL50,CH,0,0,2,Switzerland,28,0,0,69,2100,0,\N,\N
4,20220523,2022052320052048087,W24I0V5Z2L,IT,0,0,2,Italy,20,6,0,887,2100,100,20220609,20220610


In [7]:
df.shape

(900000, 16)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900000 entries, 0 to 899999
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   ymd               900000 non-null  object
 1   session_id        900000 non-null  object
 2   tracking_id       900000 non-null  object
 3   platform          900000 non-null  object
 4   is_app            900000 non-null  object
 5   is_repeater       900000 non-null  object
 6   traffic_type      900000 non-null  object
 7   country_name      900000 non-null  object
 8   agent_id          900000 non-null  object
 9   clickouts         900000 non-null  object
 10  bookings          900000 non-null  object
 11  session_duration  900000 non-null  object
 12  entry_page        900000 non-null  object
 13  total_ctp         900000 non-null  object
 14  arrival_day       900000 non-null  object
 15  departure_day     900000 non-null  object
dtypes: object(16)
memory usage: 109.9+ MB


All variables are objects, I will change the type of some of them

In [61]:
# I get all the time data about the session
df['session_timestamp'] = pd.to_datetime(df['ymd'])
df['session_year'] = df['session_timestamp'].dt.year
df['session_month'] = df['session_timestamp'].dt.month
df['session_day'] = df['session_timestamp'].dt.dayofweek # 0 is Monday, 6 is Sunday
df['sesssion_year_month'] = df['session_year'].astype(str) + '-' + df['session_month'].astype(str)


# I convert all numbers into int
df['is_app'] = df['is_app'].astype(int)
df['is_repeater'] = df['is_repeater'].astype(int)
df['clickouts'] = df['clickouts'].astype(int, errors='ignore') # there is one string value
df['traffic_type'] = df['traffic_type'].astype(int)
df['bookings'] = df['bookings'].astype(int)
df['session_duration'] = df['session_duration'].astype(int)
df['total_ctp'] = df['total_ctp'].astype(int)

# Extracting data from arrival date
df['arrival_datetime'] = pd.to_datetime(df['arrival_day'], errors='coerce')
df['arrival_year'] = df['arrival_datetime'].dt.year.astype('Int64')
df['arrival_month'] = df['arrival_datetime'].dt.month.astype('Int64')
df['arrival_weekday'] = df['arrival_datetime'].dt.dayofweek.astype('Int64')
df['arrival_year_month'] = df['arrival_year'].astype(str) + '-' + df['arrival_month'].astype(str)

# Extracting data from departing date
df['departure_daytime'] = pd.to_datetime(df['departure_day'], errors='coerce')
df['departure_year'] = df['arrival_datetime'].dt.year.astype('Int64')
df['departure_month'] = df['departure_daytime'].dt.month.astype('Int64')
df['departure_weekday'] = df['departure_daytime'].dt.dayofweek.astype('Int64')
df['departure_year_month'] = df['departure_year'].astype(str) + '-' + df['departure_month'].astype(str)

C:\Users\77479\AppData\Local\Temp\ipykernel_9400\3708900698.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['arrival_datetime'] = pd.to_datetime(df['arrival_day'], errors='coerce')
C:\Users\77479\AppData\Local\Temp\ipykernel_9400\3708900698.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['departure_daytime'] = pd.to_datetime(df['departure_day'], errors='coerce')


In [62]:
df.head()

,ymd,session_id,tracking_id,platform,is_app,is_repeater,traffic_type,country_name,agent_id,clickouts,...,arrival_datetime,arrival_year,arrival_month,arrival_weekday,arrival_year_month,departure_daytime,departure_year,departure_month,departure_weekday,departure_year_month
0,20220626,2022062620046057322,FA6JXA8TAJ,UK,0,1,2,United Kingdom,16,0,...,NaT,<NA>,<NA>,<NA>,<NA>-<NA>,NaT,<NA>,<NA>,<NA>,<NA>-<NA>
1,20220518,2022051821943006017,0X7RLU6KF7,BR,0,0,2,Brazil,2,3,...,2022-05-30,2022,5,0,2022-5,2022-05-31,2022,5,1,2022-5
2,20220508,2022050821020053928,0I59VWLQW0,UK,0,0,2,United Kingdom,20,0,...,NaT,<NA>,<NA>,<NA>,<NA>-<NA>,NaT,<NA>,<NA>,<NA>,<NA>-<NA>
3,20220507,2022050706015039122,JXNHOBQL50,CH,0,0,2,Switzerland,28,0,...,NaT,<NA>,<NA>,<NA>,<NA>-<NA>,NaT,<NA>,<NA>,<NA>,<NA>-<NA>
4,20220523,2022052320052048087,W24I0V5Z2L,IT,0,0,2,Italy,20,6,...,2022-06-09,2022,6,3,2022-6,2022-06-10,2022,6,4,2022-6


In [ ]:
df['clickouts'] = df['clickouts'].astype(int, errors )